In [7]:
import pandas as pd
import json
import boto3
import numpy as np
import textstat

<h3 style="font-family:Lora; text-align: center;"> You will need to enter both your AWS Access Key and AWS Secret Access Key</h3>

In [3]:
AWS_ACCESS_KEY = '----FILL IN----'
AWS_SECRET_ACCESS_KEY ='----FILL IN----'
AWS_REGION = 'eu-west-1'
 
client_comprehend = boto3.client(
    'comprehend',
    region_name = AWS_REGION ,
    aws_access_key_id = AWS_ACCESS_KEY,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY    
)

In [3]:
def make_df(file):
    with open(file) as data_file:    
        data = json.load(data_file)
        return pd.DataFrame(data)

In [9]:
def getReadability(movie_title, the_file, start_tweet_num, end_tweet_num):
   movie_df = make_df(the_file)
   movie_tweet_list = movie_df['text'].tolist()
   num_tweets = end_tweet_num - start_tweet_num

   lexicon_count = 0
   sentence_count = 0
   flesch_ease = 0
   flesch_kincaid = 0


   for i in movie_tweet_list[start_tweet_num: end_tweet_num]:

       lexicon_count += textstat.lexicon_count(i, removepunct=True)
       sentence_count += textstat.sentence_count(i)
       flesch_ease += textstat.flesch_reading_ease(i)
       flesch_kincaid += textstat.flesch_kincaid_grade(i)


   mean_lexicon = lexicon_count / num_tweets  # Insert num tweets instead of 50 once testing complete.
   mean_sentence = sentence_count / num_tweets
   mean_flesch = flesch_ease / num_tweets
   mean_flesch_kincaid = flesch_kincaid / num_tweets


   print('For the movie ' + movie_title + ', the mean lexicon count was ' + str(mean_lexicon) +
         ', the mean sentence count was ' + str(mean_sentence) + ', the mean flesch reading ease score was ' +
         str(mean_flesch) + ', and the mean flesch_kincaid grade level was ' + str(mean_flesch_kincaid) + '.')

getReadability('La Llorona', 'FINAL_CLEANED_DATASETS/JSON_FILES/Lallorona.json', 0, 50)

For the movie La Llorona, the mean lexicon count was 24.5, the mean sentence count was 1.56, the mean flesch reading ease score was 67.8462, and the mean flesch_kincaid grade level was 7.950000000000002.


In [4]:
def getSentiment(movie_title, the_file, start_tweet_num, end_tweet_num):
    movie_df = make_df(the_file)
    movie_tweet_list = movie_df['text'].tolist()
    num_tweets = end_tweet_num - start_tweet_num
    
    num_negative = 0
    num_positive = 0
    num_neutral = 0
    num_mixed = 0 
    
    confidence_neg_sentiment_of_tweet = 0
    confidence_pos_sentiment_of_tweet = 0
    confidence_neu_sentiment_of_tweet = 0
    confidence_mix_sentiment_of_tweet = 0
    
    for i in movie_tweet_list[start_tweet_num : end_tweet_num]:
        response_sentiment = client_comprehend.detect_sentiment(
            Text=i,
            LanguageCode='en'
        )
        sentiment = response_sentiment['Sentiment']
            
        if sentiment == 'NEGATIVE': 
            num_negative += 1
            
        elif sentiment == 'POSITIVE': 
            num_positive += 1
            
        elif sentiment == 'NEUTRAL': 
            num_neutral += 1
            
        elif sentiment == 'MIXED': 
            num_mixed += 1
            
        sentiment_score = response_sentiment['SentimentScore']
        confidence_neg_sentiment_of_tweet += sentiment_score['Negative']
        confidence_pos_sentiment_of_tweet += sentiment_score['Positive']
        confidence_neu_sentiment_of_tweet += sentiment_score['Neutral']
        confidence_mix_sentiment_of_tweet += sentiment_score['Mixed']
        
    mean_negative_confidence = confidence_neg_sentiment_of_tweet/num_tweets #Insert num tweets instead of 50 once testing complete. 
    mean_positive_confidence = confidence_pos_sentiment_of_tweet/num_tweets
    mean_neutral_confidence = confidence_neu_sentiment_of_tweet/num_tweets
    mean_mixed_confidence = confidence_mix_sentiment_of_tweet/num_tweets
    
    percent_negative = str(((num_negative / num_tweets)* 100)) + '\u0025'
    percent_positive = str(((num_positive / num_tweets) * 100)) + '\u0025'
    percent_neutral = str(((num_neutral / num_tweets) * 100)) + '\u0025'
    percent_mixed = str(((num_mixed / num_tweets) * 100)) + '\u0025'
    
    print('For the movie ' + movie_title + ', ' + percent_positive  + 
          ' of tweets were positive \u0028average confidence score was ' + str(mean_positive_confidence) + '\u0029, '
          + percent_negative + ' negative \u0028average confidence score was ' + str(mean_negative_confidence) + '\u0029,'
          + percent_neutral + ' neutral \u0028average confidence score was ' + str(mean_neutral_confidence) + '\u0029, and '
          + percent_mixed + ' mixed \u0028average confidence score was ' + str(mean_mixed_confidence) + '\u0029.')
    
getSentiment('La Llorona', 'FINAL_CLEANED_DATASETS/JSON_FILES/Lallorona.json', 0, 50)

For the movie La Llorona, 12.0% of tweets were positive (average confidence score was 0.15814473159145565), 22.0% negative (average confidence score was 0.21091976046685887),64.0% neutral (average confidence score was 0.576182504100725), and 2.0% mixed (average confidence score was 0.05475300889229402).
